In [3]:
import os
from osgeo import gdal, osr

# Define working directory
working_dir = "/mnt/f/KanoasWork/merge"

# Paths to the input files and the output files
chm_file = os.path.join(working_dir, "chm.tif")
mul_file = os.path.join(working_dir, "mul.tif")
chm_aligned = os.path.join(working_dir, "chm_aligned_clip.tif")
mul_aligned = os.path.join(working_dir, "mul_aligned.tif")
output_file = os.path.join(working_dir, "output.tif")

# Resample function
def resample_raster(input_raster, output_raster, target_epsg, resample_factor=0.5):
    # Open input raster
    input_ds = gdal.Open(input_raster)
    if input_ds is None:
        raise FileNotFoundError(f"Failed to open raster: {input_raster}")
    
    # Get geotransform and projection of the input raster
    input_geotransform = input_ds.GetGeoTransform()
    cols = input_ds.RasterXSize
    rows = input_ds.RasterYSize
    
    # Create a spatial reference object for the target EPSG
    target_srs = osr.SpatialReference()
    target_srs.ImportFromEPSG(target_epsg)
    
    # Calculate bounds manually
    min_x = input_geotransform[0]
    max_y = input_geotransform[3]
    max_x = min_x + input_geotransform[1] * cols
    min_y = max_y + input_geotransform[5] * rows
    
    # Resample and reproject using gdal.Warp
    gdal.Warp(output_raster, input_ds, 
              format='GTiff',
              dstSRS=target_srs.ExportToWkt(),
              xRes=resample_factor,
              yRes=resample_factor,
              outputBounds=(min_x, min_y, max_x, max_y),
              resampleAlg=gdal.GRA_NearestNeighbour,
              targetAlignedPixels=True)
    print(f'Resampled and saved: {output_raster}')
    input_ds = None  # Close dataset

# Run resampling for both input rasters
target_epsg = 32605  # Replace with the desired EPSG code
# resample_raster(chm_file, chm_aligned, target_epsg)
# resample_raster(mul_file, mul_aligned, target_epsg)

# Merge function
def merge_rasters(raster1, raster2, output_raster):
    # Create a temporary VRT file to stack bands
    vrt_path = os.path.splitext(output_raster)[0] + ".vrt"
    
    # Use BuildVRT to stack raster bands separately
    vrt_options = gdal.BuildVRTOptions(separate=True)
    gdal.BuildVRT(vrt_path, [raster1, raster2], options=vrt_options)
    
    # Translate the VRT to a single output file in GeoTIFF format
    translate_options = gdal.TranslateOptions(format='GTiff', noData=-9999)
    gdal.Translate(output_raster, vrt_path, options=translate_options)
    
    # Clean up the VRT file
    os.remove(vrt_path)

# Merge the resampled rasters
merge_rasters(mul_aligned, chm_aligned, output_file)

# Check band count of the final output
merged_ds = gdal.Open(output_file)
if merged_ds:
    print(f"Merged output saved to: {output_file}")
    print("Number of bands in the merged output:", merged_ds.RasterCount)
    merged_ds = None  # Close dataset
else:
    print("Failed to create merged output.")


Warning 1: output.tif: Unable to export GeoTIFF file with different datatypes per different bands. All bands should have the same types in TIFF.


Merged output saved to: /mnt/f/KanoasWork/merge/output.tif
Number of bands in the merged output: 9
